In [23]:
import os
from dotenv import load_dotenv
import json
from apify_client import ApifyClient
import duckdb as db
import pandas as pd
from datetime import datetime

# Conexões do DuckDB com extensão JSON
con = db.connect()
con.install_extension('json')
con.load_extension('json')

# Define a data atual
hoje = datetime.now().strftime('%Y%m%d')

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [24]:
# Instala o python-dotenv se não estiver instalado
try:
    import dotenv
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])

In [ ]:
def tratar_link_insta(link):
    user = str(link).split('/')[3]
    return user

In [12]:
df1 = pd.read_excel('Perfis testes - Novembro.xlsx', sheet_name='Hyeser')
df2 = pd.read_excel('Perfis testes - Novembro.xlsx', sheet_name='Fabio')
df2 = df2[['LINK', 'Rede']]
df2 = df2.query("Rede == 'Instagram'")
df = pd.concat([df1, df2], ignore_index=True)
df['LINK'] = df['LINK'].apply(lambda x: tratar_link_insta(x))
lista_usernames = list(df['LINK'])
max_users = int(len(lista_usernames))
max_users

59

In [28]:
# Initialize the ApifyClient with your API token from environment variable
client = ApifyClient(os.getenv("APIFY_API_TOKEN"))

# Prepare the Actor input
run_input = {
    "usernames": lista_usernames,
    "maxUsernames": max_users,
    "delayBetweenRequests": 2000,
}

# Run the Actor and wait for it to finish
run = client.actor("7eNb0Yr5dLwy1jfPS").call(run_input=run_input)

# # Fetch and print Actor results from the run's dataset (if there are any)
# for item in client.dataset(run["defaultDatasetId"]).iterate_items():
#     print(item)

[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> Status: RUNNING, Message: 
[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> 2025-11-07T19:59:51.912Z ACTOR: Pulling container image of build 6SdYWbsdK1dAUtt2P from registry.
[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> 2025-11-07T19:59:52.395Z ACTOR: Creating container.
[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> 2025-11-07T19:59:52.436Z ACTOR: Starting container.
[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> 2025-11-07T19:59:53.445Z INFO  System info {"apifyVersion":"3.5.0","apifyClientVersion":"2.17.0","crawleeVersion":"3.15.1","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> 2025-11-07T19:59:57.344Z ✅ Stories checked for @smsindelicado → stories saved
[apify.instagram-story-viewer runId:YMrbmsfoym0CAwxAG] -> 2025-11-07T20:00:01.412Z ✅ Stories checked for @espetacular → stories saved
[apify.instagram-story-viewer runId:YMrbmsfoy

In [ ]:
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    with open(f'teste_json/output_{item["requested"]}.json', 'a', encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False, indent=4)
        f.write('\n')

In [10]:
for item in os.listdir('teste_json'):
    if item.endswith('.json'):
        with open(os.path.join('teste_json', item), 'r', encoding='utf-8') as f:
            data = f.read()
            data = json.loads(data)
            f.close()

        if data['stories'] == 'No story available':
            os.rename(os.path.join('teste_json', item), os.path.join('teste_json_sem_stories', item))

In [17]:
path = r'teste_json\*.json'
df = con.execute('''
with parte1 as (
SELECT
requested as username,
UNNEST(stories) as stories
FROM read_json_auto(?, ignore_errors=true)
),
parte2 as (
SELECT
username,
UNNEST(stories.story_link_stickers).story_link.display_url as story_link_url
FROM parte1
)   
SELECT username,
story_link_url,
case
when split(story_link_url, '/')[1] = 'amzlink.to' then 'Amazon'
when split(story_link_url, '/')[1] = 'mercadolivre.com' then 'Mercado Livre'
when split(story_link_url, '/')[1] = 's.shopee.com.br' then 'Shopee'
when split(story_link_url, '/')[1] = 'minhaloja.natura.com' then 'Natura'
when split(story_link_url, '/')[1] = 'magazinevoce.com.br' then 'Magazine Luiza'
when split(story_link_url, '/')[1] = 'elausa.com.br' then 'Ela Usa'
when split(story_link_url, '/')[1] = 'epocacosmeticos.com.br' then 'Época Cosméticos'
when split(story_link_url, '/')[1] = 'natura.com.br' then 'Natura'
when split(story_link_url, '/')[1] = 'sminhaloja.natura.com' then 'Natura'
when split(story_link_url, '/')[1] = 'api.whatsapp.com' then 'WhatsApp'
when split(story_link_url, '/')[1] = 'google.com' then 'Google'
when split(story_link_url, '/')[1] = 'encurtador.com.br' then 'Encurtador'
else null end as origin
FROM parte2
''', [path]).df()
df.to_csv('output_final.csv', index=False)

In [ ]:
df_val = pd.read_csv(f'output_final_{hoje}.csv')
df_val.username.nunique()

52